# Torch Core

This module contains all the basic functions we need in other modules of the fastai library (split with [`core`](/core.html#core) that contains the ones not requiring pytorch). Its documentation can easily be skipped at a first read, unless you want to know what a given function does.

In [ ]:
from fastai.imports import *
from fastai.gen_doc.nbdoc import *
from fastai.layers import *
from fastai.torch_core import * 

## Global constants

`AdamW = partial(optim.Adam, betas=(0.9,0.99))` <div style="text-align: right"><a href="https://github.com/fastai/fastai/blob/master/fastai/torch_core.py#L43">[source]</a></div>

`bn_types = (nn.BatchNorm1d, nn.BatchNorm2d, nn.BatchNorm3d)` <div style="text-align: right"><a href="https://github.com/fastai/fastai/blob/master/fastai/torch_core.py#L41">[source]</a></div>

`defaults.device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')` <div style="text-align: right"><a href="https://github.com/fastai/fastai/blob/master/fastai/torch_core.py#L62">[source]</a></div>

If you are trying to make fastai run on the CPU, simply change the default device: `defaults.device = 'cpu'`. 

Alternatively, if not using wildcard imports: `fastai.torch_core.defaults.device = 'cpu'`.

## Functions that operate conversions

In [ ]:
show_doc(batch_to_half)

<h4 id="batch_to_half"><code>batch_to_half</code><a href="https://github.com/fastai/fastai/blob/master/fastai/torch_core.py#L203" class="source_link">[source]</a></h4>

> <code>batch_to_half</code>(**`b`**:`Collection`\[`Tensor`\]) → `Collection`\[`Tensor`\]

Set the input of batch `b` to half precision.  

In [ ]:
show_doc(flatten_model, full_name='flatten_model')

<h4 id="flatten_model"><code>flatten_model</code><a href="https://github.com/fastai/fastai/blob/master/fastai/torch_core.py#L157" class="source_link">[source]</a></h4>

> <code>flatten_model</code>(**`m`**)

Flattens all the layers of `m` into an array. This allows for easy access to the layers of the model and allows you to manipulate the model as if it was an array.

In [ ]:
m = simple_cnn([3,6,12])
m

Sequential(
  (0): Sequential(
    (0): Conv2d(3, 6, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (1): ReLU(inplace)
  )
  (1): Sequential(
    (0): Conv2d(6, 12, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (1): ReLU(inplace)
  )
  (2): Sequential(
    (0): AdaptiveAvgPool2d(output_size=1)
    (1): Flatten()
  )
)

In [ ]:
flatten_model(m)

[Conv2d(3, 6, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1)),
 ReLU(inplace),
 Conv2d(6, 12, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1)),
 ReLU(inplace),
 AdaptiveAvgPool2d(output_size=1),
 Flatten()]

In [ ]:
show_doc(model2half)

<h4 id="model2half"><code>model2half</code><a href="https://github.com/fastai/fastai/blob/master/fastai/torch_core.py#L213" class="source_link">[source]</a></h4>

> <code>model2half</code>(**`model`**:[`Module`](https://pytorch.org/docs/stable/nn.html#torch.nn.Module)) → [`Module`](https://pytorch.org/docs/stable/nn.html#torch.nn.Module)

Convert `model` to half precision except the batchnorm layers.  

Converting model parameters to half precision allows us to leverage fast `FP16` arithmetic which can speed up the computations by 2-8 times. It also reduces memory consumption allowing us to train deeper models. 

**Note**: Batchnorm layers are not converted to half precision as that may lead to instability in training.

In [ ]:
m = simple_cnn([3,6,12], bn=True)

def show_params_dtype(state_dict):
    """Simple function to pretty print the dtype of the model params"""
    for wt_name, param in state_dict.items():
        print("{:<30}: {}".format(wt_name, str(param.dtype)))
    print()    

print("dtypes of model parameters before model2half: ")
show_params_dtype(m.state_dict())

# Converting model to half precision
m_half = model2half(m)

print("dtypes of model parameters after model2half: ")
show_params_dtype(m_half.state_dict())

dtypes of model parameters before model2half: 
0.0.weight                    : torch.float32
0.2.weight                    : torch.float32
0.2.bias                      : torch.float32
0.2.running_mean              : torch.float32
0.2.running_var               : torch.float32
0.2.num_batches_tracked       : torch.int64
1.0.weight                    : torch.float32
1.0.bias                      : torch.float32

dtypes of model parameters after model2half: 
0.0.weight                    : torch.float16
0.2.weight                    : torch.float32
0.2.bias                      : torch.float32
0.2.running_mean              : torch.float32
0.2.running_var               : torch.float32
0.2.num_batches_tracked       : torch.int64
1.0.weight                    : torch.float16
1.0.bias                      : torch.float16



In [ ]:
show_doc(np2model_tensor)

<h4 id="np2model_tensor"><code>np2model_tensor</code><a href="https://github.com/fastai/fastai/blob/master/fastai/torch_core.py#L260" class="source_link">[source]</a></h4>

> <code>np2model_tensor</code>(**`a`**)

Tranform numpy array `a` to a tensor of the same type.  

It is a wrapper on top of Pytorch's `torch.as_tensor` which converts numpy array to torch tensor, and additionally attempts to map all floats to `torch.float32` and all integers to `torch.int64` for consistencies in model data. Below is an example demonstrating it's functionality for floating number, similar functionality applies to integer as well.

In [ ]:
a1 = np.ones((2, 3)).astype(np.float16)
a2 = np.ones((2, 3)).astype(np.float32)
a3 = np.ones((2, 3)).astype(np.float64)

b1 = np2model_tensor(a1) # Maps to torch.float32
b2 = np2model_tensor(a2) # Maps to torch.float32
b3 = np2model_tensor(a3) # Maps to torch.float32

print(f"Datatype of as': {a1.dtype}, {a2.dtype}, {a3.dtype}")
print(f"Datatype of bs': {b1.dtype}, {b2.dtype}, {b3.dtype}")

Datatype of as': float16, float32, float64
Datatype of bs': torch.float32, torch.float32, torch.float32


In [ ]:
show_doc(requires_grad)

<h4 id="requires_grad"><code>requires_grad</code><a href="https://github.com/fastai/fastai/blob/master/fastai/torch_core.py#L117" class="source_link">[source]</a></h4>

> <code>requires_grad</code>(**`m`**:[`Module`](https://pytorch.org/docs/stable/nn.html#torch.nn.Module), **`b`**:`Optional`\[`bool`\]=***`None`***) → `Optional`\[`bool`\]

If `b` is not set return [`requires_grad`](/torch_core.html#requires_grad) of first param, else set [`requires_grad`](/torch_core.html#requires_grad) on all params as `b`  

Performs both getting and setting of [`requires_grad`](/torch_core.html#requires_grad) parameter of the tensors, which decided whether to accumulate gradients or not. 

* If `b` is `None`: The function **gets** the [`requires_grad`](/torch_core.html#requires_grad) for the model parameter, to be more specific it returns the [`requires_grad`](/torch_core.html#requires_grad) of the first element in the model.

* Else if `b` is passed (a boolean value), [`requires_grad`](/torch_core.html#requires_grad) of all parameters of the model is **set** to `b`.

In [ ]:
# Any Pytorch model
m = simple_cnn([3, 6, 12], bn=True)

# Get the requires_grad of model
print("requires_grad of model: {}".format(requires_grad(m)))

# Set requires_grad of all params in model to false
requires_grad(m, False)

# Get the requires_grad of model
print("requires_grad of model: {}".format(requires_grad(m)))

requires_grad of model: True
requires_grad of model: False


In [ ]:
show_doc(tensor)

<h4 id="tensor"><code>tensor</code><a href="https://github.com/fastai/fastai/blob/master/fastai/torch_core.py#L65" class="source_link">[source]</a></h4>

> <code>tensor</code>(**`x`**:`Any`, **\*`rest`**) → `Tensor`

Like `torch.as_tensor`, but handle lists too, and can pass multiple vector elements directly.  

Handy function when you want to convert any list type object to tensor, initialize your weights manually, and other similar cases.

**NB**: When passing multiple vectors, all vectors must be of same dimensions. (Obvious but can be forgotten sometimes)

In [ ]:
# Conversion from any numpy array
b = tensor(np.array([1, 2, 3]))
print(b, type(b))

# Passing as multiple parameters
b = tensor(1, 2, 3)
print(b, type(b))

# Passing a single list
b = tensor([1, 2, 3])
print(b, type(b))

# Can work with multiple vectors / lists
b = tensor([1, 2], [3, 4])
print(b, type(b))

tensor([1, 2, 3]) <class 'torch.Tensor'>
tensor([1, 2, 3]) <class 'torch.Tensor'>
tensor([1, 2, 3]) <class 'torch.Tensor'>
tensor([[1, 2],
        [3, 4]]) <class 'torch.Tensor'>


In [ ]:
show_doc(to_cpu)

<h4 id="to_cpu"><code>to_cpu</code><a href="https://github.com/fastai/fastai/blob/master/fastai/torch_core.py#L92" class="source_link">[source]</a></h4>

> <code>to_cpu</code>(**`b`**:`ItemsList`)

Recursively map lists of tensors in `b ` to the cpu.  

A wrapper on top of Pytorch's `torch.Tensor.cpu()` function, which creates and returns a copy of a tensor or even a **list** of tensors in the CPU. As described in Pytorch's docs, if the tensor or list of tensor is already on the CPU, the exact data is returned and no copy is made.

Useful to convert all the list of parameters of the model to CPU in a single call.

In [ ]:
if torch.cuda.is_available():
    a = [torch.randn((1, 1)).cuda() for i in range(3)]
    print(a)
    print("Id of tensors in a: ")
    for i in a: print(id(i))
    
    # Getting a CPU version of the tensors in GPU
    b = to_cpu(a)
    print(b)
    print("Id of tensors in b:")
    for i in b: print(id(i))
    
    # Trying to perform to_cpu on a list of tensor already in CPU
    c = to_cpu(b)
    print(c)
    # The tensors in c has exact id as that of b. No copy performed.
    print("Id of tensors in c:")
    for i in c: print(id(i))

[tensor([[-0.5932]], device='cuda:0'), tensor([[-0.2867]], device='cuda:0'), tensor([[-1.0616]], device='cuda:0')]
Id of tensors in a: 
139974954993416
139977016149120
139974955521008
[tensor([[-0.5932]]), tensor([[-0.2867]]), tensor([[-1.0616]])]
Id of tensors in b:
139974954963016
139974955458280
139974955521152
[tensor([[-0.5932]]), tensor([[-0.2867]]), tensor([[-1.0616]])]
Id of tensors in c:
139974954963016
139974955458280
139974955521152


In [ ]:
show_doc(to_data)

<h4 id="to_data"><code>to_data</code><a href="https://github.com/fastai/fastai/blob/master/fastai/torch_core.py#L87" class="source_link">[source]</a></h4>

> <code>to_data</code>(**`b`**:`ItemsList`)

Recursively map lists of items in `b ` to their wrapped data.  

Returns the data attribute from the object or collection of objects that inherits from [`ItemBase`](/core.html#ItemBase) class. Useful to examine the exact values of the data, could be used to work with the data outside of `fastai` classes.

In [ ]:
# Default example examined

from fastai import *
from fastai.vision import *

path = untar_data(URLs.MNIST_SAMPLE)
data = ImageDataBunch.from_folder(path)

# Examin the labels
ys = list(data.y)
print("Category display names: ", [ys[0], ys[-1]])

print("Unique classes internally represented as: ", to_data([ys[0], ys[-1]]))

Category display names:  [Category 3, Category 7]
Unique classes internally represented as:  [0, 1]


In [ ]:
show_doc(to_detach)

<h4 id="to_detach"><code>to_detach</code><a href="https://github.com/fastai/fastai/blob/master/fastai/torch_core.py#L80" class="source_link">[source]</a></h4>

> <code>to_detach</code>(**`b`**:`Tensors`, **`cpu`**:`bool`=***`True`***)

Recursively detach lists of tensors in `b `; put them on the CPU if `cpu=True`.  

In [ ]:
show_doc(to_device)

<h4 id="to_device"><code>to_device</code><a href="https://github.com/fastai/fastai/blob/master/fastai/torch_core.py#L107" class="source_link">[source]</a></h4>

> <code>to_device</code>(**`b`**:`Tensors`, **`device`**:[`device`](https://pytorch.org/docs/stable/tensor_attributes.html#torch-device))

Recursively put `b` on `device`.  

In [ ]:
show_doc(to_half)

<h4 id="to_half"><code>to_half</code><a href="https://github.com/fastai/fastai/blob/master/fastai/torch_core.py#L97" class="source_link">[source]</a></h4>

> <code>to_half</code>(**`b`**:`Collection`\[`Tensor`\]) → `Collection`\[`Tensor`\]

Recursively map lists of tensors in `b ` to FP16.  

Converts the tensor or list of `FP16`, resulting in less memory consumption and faster computations with the tensor. It does not convert `torch.int` types to half precision.

In [ ]:
a1 = torch.tensor([1, 2], dtype=torch.int64)
a2 = torch.tensor([1, 2], dtype=torch.int32)
a3 = torch.tensor([1, 2], dtype=torch.int16)
a4 = torch.tensor([1, 2], dtype=torch.float64)
a5 = torch.tensor([1, 2], dtype=torch.float32)
a6 = torch.tensor([1, 2], dtype=torch.float16)

print("dtype of as: ", a1.dtype, a2.dtype, a3.dtype, a4.dtype, a5.dtype, a6.dtype, sep="\t")

b1, b2, b3, b4, b5, b6 = to_half([a1, a2, a3, a4, a5, a6])

print("dtype of bs: ", b1.dtype, b2.dtype, b3.dtype, b4.dtype, b5.dtype, b6.dtype, sep="\t")

dtype of as: 	torch.int64	torch.int32	torch.int16	torch.float64	torch.float32	torch.float16
dtype of bs: 	torch.int64	torch.int32	torch.int16	torch.float16	torch.float16	torch.float16


In [ ]:
show_doc(to_np)

<h4 id="to_np"><code>to_np</code><a href="https://github.com/fastai/fastai/blob/master/fastai/torch_core.py#L278" class="source_link">[source]</a></h4>

> <code>to_np</code>(**`x`**)

Convert a tensor to a numpy array.  

Internally puts the data to CPU, and converts to `numpy.ndarray` equivalent of `torch.tensor` by calling `torch.Tensor.numpy()`.

In [ ]:
a = torch.tensor([1, 2], dtype=torch.float64)

if torch.cuda.is_available():
    a = a.cuda()

print(a, type(a), a.device)

b = to_np(a)

print(b, type(b))

tensor([1., 2.], dtype=torch.float64) <class 'torch.Tensor'> cpu
[1. 2.] <class 'numpy.ndarray'>


In [ ]:
show_doc(try_int)

<h4 id="try_int"><code>try_int</code><a href="https://github.com/fastai/fastai/blob/master/fastai/torch_core.py#L327" class="source_link">[source]</a></h4>

> <code>try_int</code>(**`o`**:`Any`) → `Any`

Try to convert `o` to int, default to `o` if not possible.  

In [ ]:
# Converts floating point numbers to integer
print(try_int(12.5), type(try_int(12.5)))

# This is a Rank-1 ndarray, which ideally should not be converted to int 
print(try_int(np.array([1.5])), try_int(np.array([1.5])).dtype)

# Numpy array with a single elements are converted to int
print(try_int(np.array(1.5)), type(try_int(np.array(1.5))))

print(try_int(torch.tensor(2.5)), type(try_int(torch.tensor(2.5))))

# Strings are not converted to int (of course)
print(try_int("12.5"), type(try_int("12.5")))

12 <class 'int'>
[1.5] float64
1 <class 'int'>
2 <class 'int'>
12.5 <class 'str'>


## Functions to deal with model initialization

In [ ]:
show_doc(apply_init)

<h4 id="apply_init"><code>apply_init</code><a href="https://github.com/fastai/fastai/blob/master/fastai/torch_core.py#L234" class="source_link">[source]</a></h4>

> <code>apply_init</code>(**`m`**, **`init_func`**:`LayerFunc`)

Initialize all non-batchnorm layers of `m` with `init_func`.  

In [ ]:
show_doc(apply_leaf)

<h4 id="apply_leaf"><code>apply_leaf</code><a href="https://github.com/fastai/fastai/blob/master/fastai/torch_core.py#L228" class="source_link">[source]</a></h4>

> <code>apply_leaf</code>(**`m`**:[`Module`](https://pytorch.org/docs/stable/nn.html#torch.nn.Module), **`f`**:`LayerFunc`)

Apply `f` to children of `m`.  

In [ ]:
show_doc(cond_init)

<h4 id="cond_init"><code>cond_init</code><a href="https://github.com/fastai/fastai/blob/master/fastai/torch_core.py#L224" class="source_link">[source]</a></h4>

> <code>cond_init</code>(**`m`**:[`Module`](https://pytorch.org/docs/stable/nn.html#torch.nn.Module), **`init_func`**:`LayerFunc`)

Initialize the non-batchnorm layers of `m` with `init_func`.  

In [ ]:
show_doc(in_channels)

<h4 id="in_channels"><code>in_channels</code><a href="https://github.com/fastai/fastai/blob/master/fastai/torch_core.py#L238" class="source_link">[source]</a></h4>

> <code>in_channels</code>(**`m`**:[`Module`](https://pytorch.org/docs/stable/nn.html#torch.nn.Module)) → `List`\[`int`\]

Return the shape of the first weight layer in `m`.  

In [ ]:
show_doc(init_default)

<h4 id="init_default"><code>init_default</code><a href="https://github.com/fastai/fastai/blob/master/fastai/torch_core.py#L217" class="source_link">[source]</a></h4>

> <code>init_default</code>(**`m`**:[`Module`](https://pytorch.org/docs/stable/nn.html#torch.nn.Module), **`func`**:`LayerFunc`=***`'kaiming_normal_'`***)

Initialize `m` weights with `func` and set `bias` to 0.  

## Functions to get information of a model

In [ ]:
show_doc(children)

<h4 id="children"><code>children</code><a href="https://github.com/fastai/fastai/blob/master/fastai/torch_core.py#L129" class="source_link">[source]</a></h4>

> <code>children</code>(**`m`**:[`Module`](https://pytorch.org/docs/stable/nn.html#torch.nn.Module)) → `ModuleList`

Get children of `m`.  

In [ ]:
show_doc(children_and_parameters)

<h4 id="children_and_parameters"><code>children_and_parameters</code><a href="https://github.com/fastai/fastai/blob/master/fastai/torch_core.py#L149" class="source_link">[source]</a></h4>

> <code>children_and_parameters</code>(**`m`**:[`Module`](https://pytorch.org/docs/stable/nn.html#torch.nn.Module))

Return the children of `m` and its direct parameters not registered in modules.  

In [ ]:
show_doc(first_layer)

<h4 id="first_layer"><code>first_layer</code><a href="https://github.com/fastai/fastai/blob/master/fastai/torch_core.py#L159" class="source_link">[source]</a></h4>

> <code>first_layer</code>(**`m`**:[`Module`](https://pytorch.org/docs/stable/nn.html#torch.nn.Module)) → [`Module`](https://pytorch.org/docs/stable/nn.html#torch.nn.Module)

Retrieve first layer in a module `m`.  

In [ ]:
show_doc(last_layer)

<h4 id="last_layer"><code>last_layer</code><a href="https://github.com/fastai/fastai/blob/master/fastai/torch_core.py#L163" class="source_link">[source]</a></h4>

> <code>last_layer</code>(**`m`**:[`Module`](https://pytorch.org/docs/stable/nn.html#torch.nn.Module)) → [`Module`](https://pytorch.org/docs/stable/nn.html#torch.nn.Module)

Retrieve last layer in a module `m`.  

In [ ]:
show_doc(num_children)

<h4 id="num_children"><code>num_children</code><a href="https://github.com/fastai/fastai/blob/master/fastai/torch_core.py#L133" class="source_link">[source]</a></h4>

> <code>num_children</code>(**`m`**:[`Module`](https://pytorch.org/docs/stable/nn.html#torch.nn.Module)) → `int`

Get number of children modules in `m`.  

In [ ]:
show_doc(one_param)

<h4 id="one_param"><code>one_param</code><a href="https://github.com/fastai/fastai/blob/master/fastai/torch_core.py#L323" class="source_link">[source]</a></h4>

> <code>one_param</code>(**`m`**:[`Module`](https://pytorch.org/docs/stable/nn.html#torch.nn.Module)) → `Tensor`

Return the first parameter of `m`.  

In [ ]:
show_doc(range_children)

<h4 id="range_children"><code>range_children</code><a href="https://github.com/fastai/fastai/blob/master/fastai/torch_core.py#L137" class="source_link">[source]</a></h4>

> <code>range_children</code>(**`m`**:[`Module`](https://pytorch.org/docs/stable/nn.html#torch.nn.Module)) → `Iterator`\[`int`\]

Return iterator of len of children of `m`.  

In [ ]:
show_doc(trainable_params)

<h4 id="trainable_params"><code>trainable_params</code><a href="https://github.com/fastai/fastai/blob/master/fastai/torch_core.py#L124" class="source_link">[source]</a></h4>

> <code>trainable_params</code>(**`m`**:[`Module`](https://pytorch.org/docs/stable/nn.html#torch.nn.Module)) → `ParamList`

Return list of trainable params in `m`.  

## Functions to deal with BatchNorm layers

In [ ]:
show_doc(bn2float)

<h4 id="bn2float"><code>bn2float</code><a href="https://github.com/fastai/fastai/blob/master/fastai/torch_core.py#L207" class="source_link">[source]</a></h4>

> <code>bn2float</code>(**`module`**:[`Module`](https://pytorch.org/docs/stable/nn.html#torch.nn.Module)) → [`Module`](https://pytorch.org/docs/stable/nn.html#torch.nn.Module)

If `module` is batchnorm don't use half precision.  

In [ ]:
show_doc(set_bn_eval)

<h4 id="set_bn_eval"><code>set_bn_eval</code><a href="https://github.com/fastai/fastai/blob/master/fastai/torch_core.py#L196" class="source_link">[source]</a></h4>

> <code>set_bn_eval</code>(**`m`**:[`Module`](https://pytorch.org/docs/stable/nn.html#torch.nn.Module))

Set bn layers in eval mode for all recursive children of `m`.  

In [ ]:
show_doc(split_bn_bias)

<h4 id="split_bn_bias"><code>split_bn_bias</code><a href="https://github.com/fastai/fastai/blob/master/fastai/torch_core.py#L185" class="source_link">[source]</a></h4>

> <code>split_bn_bias</code>(**`layer_groups`**:`ModuleList`) → `ModuleList`

Split the layers in `layer_groups` into batchnorm (`bn_types`) and non-batchnorm groups.  

## Functions to get random tensors

In [ ]:
show_doc(log_uniform)

<h4 id="log_uniform"><code>log_uniform</code><a href="https://github.com/fastai/fastai/blob/master/fastai/torch_core.py#L310" class="source_link">[source]</a></h4>

> <code>log_uniform</code>(**`low`**, **`high`**, **`size`**:`Optional`\[`List`\[`int`\]\]=***`None`***) → `FloatOrTensor`

Draw 1 or shape=`size` random floats from uniform dist: min=log(`low`), max=log(`high`).  

In [ ]:
log_uniform(0.5,2,(8,))

tensor([0.5775, 0.7902, 0.6087, 0.5730, 0.8057, 0.8845, 0.8975, 0.5585])

In [ ]:
show_doc(rand_bool)

<h4 id="rand_bool"><code>rand_bool</code><a href="https://github.com/fastai/fastai/blob/master/fastai/torch_core.py#L315" class="source_link">[source]</a></h4>

> <code>rand_bool</code>(**`p`**:`float`, **`size`**:`Optional`\[`List`\[`int`\]\]=***`None`***) → `BoolOrTensor`

Draw 1 or shape=`size` random booleans (`True` occuring with probability `p`).  

In [ ]:
rand_bool(0.5, 8)

tensor([1, 1, 0, 1, 0, 0, 1, 0], dtype=torch.uint8)

In [ ]:
show_doc(uniform)

<h4 id="uniform"><code>uniform</code><a href="https://github.com/fastai/fastai/blob/master/fastai/torch_core.py#L305" class="source_link">[source]</a></h4>

> <code>uniform</code>(**`low`**:`Number`, **`high`**:`Number`=***`None`***, **`size`**:`Optional`\[`List`\[`int`\]\]=***`None`***) → `FloatOrTensor`

Draw 1 or shape=`size` random floats from uniform dist: min=`low`, max=`high`.  

In [ ]:
uniform(0,1,(8,))

tensor([0.6432, 0.3110, 0.7588, 0.7058, 0.7121, 0.8552, 0.3352, 0.2620])

In [ ]:
show_doc(uniform_int)

<h4 id="uniform_int"><code>uniform_int</code><a href="https://github.com/fastai/fastai/blob/master/fastai/torch_core.py#L319" class="source_link">[source]</a></h4>

> <code>uniform_int</code>(**`low`**:`int`, **`high`**:`int`, **`size`**:`Optional`\[`List`\[`int`\]\]=***`None`***) → `IntOrTensor`

Generate int or tensor `size` of ints between `low` and `high` (included).  

In [ ]:
uniform_int(0,2,(8,))

tensor([0, 1, 1, 2, 1, 1, 1, 2])

## Other functions

In [ ]:
show_doc(ParameterModule, title_level=3)

<h3 id="ParameterModule"><code>class</code> <code>ParameterModule</code><a href="https://github.com/fastai/fastai/blob/master/fastai/torch_core.py#L141" class="source_link">[source]</a></h3>

> <code>ParameterModule</code>(**`p`**:[`Parameter`](https://pytorch.org/docs/stable/nn.html#torch.nn.Parameter)) :: [`Module`](https://pytorch.org/docs/stable/nn.html#torch.nn.Module)

Register a lone parameter `p` in a module.  

In [ ]:
show_doc(calc_loss)

<h4 id="calc_loss"><code>calc_loss</code><a href="https://github.com/fastai/fastai/blob/master/fastai/torch_core.py#L244" class="source_link">[source]</a></h4>

> <code>calc_loss</code>(**`y_pred`**:`Tensor`, **`y_true`**:`Tensor`, **`loss_func`**:`LossFunction`)

Calculate loss between `y_pred` and `y_true` using `loss_func`.  

In [ ]:
show_doc(data_collate)

<h4 id="data_collate"><code>data_collate</code><a href="https://github.com/fastai/fastai/blob/master/fastai/torch_core.py#L113" class="source_link">[source]</a></h4>

> <code>data_collate</code>(**`batch`**:`ItemsList`) → `Tensor`

Convert `batch` items to tensor data.  

In [ ]:
show_doc(get_model)

<h4 id="get_model"><code>get_model</code><a href="https://github.com/fastai/fastai/blob/master/fastai/torch_core.py#L335" class="source_link">[source]</a></h4>

> <code>get_model</code>(**`model`**:[`Module`](https://pytorch.org/docs/stable/nn.html#torch.nn.Module))

Return the model maybe wrapped inside `model`.  

In [ ]:
show_doc(grab_idx)

<h4 id="grab_idx"><code>grab_idx</code><a href="https://github.com/fastai/fastai/blob/master/fastai/torch_core.py#L290" class="source_link">[source]</a></h4>

> <code>grab_idx</code>(**`x`**, **`i`**, **`batch_first`**:`bool`=***`True`***)

Grab the `i`-th batch in `x`, `batch_first` stating the batch dimension.  

In [ ]:
show_doc(logit)

<h4 id="logit"><code>logit</code><a href="https://github.com/fastai/fastai/blob/master/fastai/torch_core.py#L295" class="source_link">[source]</a></h4>

> <code>logit</code>(**`x`**:`Tensor`) → `Tensor`

Logit of `x`, clamped to avoid inf.  

In [ ]:
show_doc(logit_)

<h4 id="logit_"><code>logit_</code><a href="https://github.com/fastai/fastai/blob/master/fastai/torch_core.py#L300" class="source_link">[source]</a></h4>

> <code>logit_</code>(**`x`**:`Tensor`) → `Tensor`

Inplace logit of `x`, clamped to avoid inf  

In [ ]:
show_doc(model_type)

<h4 id="model_type"><code>model_type</code><a href="https://github.com/fastai/fastai/blob/master/fastai/torch_core.py#L254" class="source_link">[source]</a></h4>

> <code>model_type</code>(**`dtype`**)

Return the torch type corresponding to `dtype`.  

In [ ]:
show_doc(np_address)

<h4 id="np_address"><code>np_address</code><a href="https://github.com/fastai/fastai/blob/master/fastai/torch_core.py#L76" class="source_link">[source]</a></h4>

> <code>np_address</code>(**`x`**:`ndarray`) → `int`

Address of `x` in memory.  

In [ ]:
show_doc(split_model)

<h4 id="split_model"><code>split_model</code><a href="https://github.com/fastai/fastai/blob/master/fastai/torch_core.py#L174" class="source_link">[source]</a></h4>

> <code>split_model</code>(**`model`**:[`Module`](https://pytorch.org/docs/stable/nn.html#torch.nn.Module), **`splits`**:`Collection`\[`Union`\[[`Module`](https://pytorch.org/docs/stable/nn.html#torch.nn.Module), `ModuleList`\]\], **`want_idxs`**:`bool`=***`False`***)

Split `model` according to the layers in `splits`.  

If `splits` are layers, the model is split at those (not included) sequentially. If `want_idxs` is True, the corresponding indexes are returned. If `splits` are lists of layers, the model is split according to those.

In [ ]:
show_doc(split_model_idx)

<h4 id="split_model_idx"><code>split_model_idx</code><a href="https://github.com/fastai/fastai/blob/master/fastai/torch_core.py#L167" class="source_link">[source]</a></h4>

> <code>split_model_idx</code>(**`model`**:[`Module`](https://pytorch.org/docs/stable/nn.html#torch.nn.Module), **`idxs`**:`Collection`\[`int`\]) → `ModuleList`

Split `model` according to the indexes in `idxs`.  

In [ ]:
show_doc(trange_of)

<h4 id="trange_of"><code>trange_of</code><a href="https://github.com/fastai/fastai/blob/master/fastai/torch_core.py#L274" class="source_link">[source]</a></h4>

> <code>trange_of</code>(**`x`**)

Create a tensor from `range_of(x)`.  

## Undocumented Methods - Methods moved below this line will intentionally be hidden

In [ ]:
show_doc(tensor__array__)

<h4 id="tensor__array__"><code>tensor__array__</code><a href="https://github.com/fastai/fastai/blob/master/fastai/torch_core.py#L283" class="source_link">[source]</a></h4>

> <code>tensor__array__</code>(**`dtype`**=***`None`***)

In [ ]:
show_doc(ParameterModule.forward)

<h4 id="ParameterModule.forward"><code>forward</code><a href="https://github.com/fastai/fastai/blob/master/fastai/torch_core.py#L147" class="source_link">[source]</a></h4>

> <code>forward</code>(**`x`**)

Defines the computation performed at every call. Should be overridden by all subclasses.

.. note::
    Although the recipe for forward pass needs to be defined within
    this function, one should call the :class:`Module` instance afterwards
    instead of this since the former takes care of running the
    registered hooks while the latter silently ignores them. 

## New Methods - Please document or move to the undocumented section